In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re

def csv_files(path):
    """Função usada para ler os ficheiros que tem o simbolo do Euro"""
    import csv
    f = open(path, encoding='windows-1252', errors='ignore')
    data = []
    for row in csv.reader(f, delimiter=','):
        data.append(row)
    col = [*data[0]]
    data.pop(0)
    df = pd.DataFrame(data, columns=col)
    return df, col

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value


In [3]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'code'
tw_doer = 'date_of_equipment_removal'
tw_status = 'site status'
tw_bts = 'bts sites'
tw_bill = 'rfai ( ready for active installation )'
tw_wip = 'wip_site'
tw_decom = 'decommissioned sites'
tw_critical = 'critical site'

msa_index = 'code'
msa_doer = 'date_of_equipment_removal'
msa_status = 'site status'
msa_bts = 'bts sites'
msa_bill = 'rfai ( ready for active installation )'
msa_wip = 'wip_site'
msa_decom = 'decommissioned sites'
msa_critical = 'critical site'

In [45]:
def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        #newlist = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), columns))
        #newlist = list(map(lambda x: re.sub(r'\s+$', '', str(x.lower())), columns))        
        return newlist

towerdb = pd.read_excel('/content/TowerDB_Romania_20210831 (1).xlsx', sheet_name='TowerDB')
towerdb = towerdb.iloc[5:,:]
towerdb.columns = lower_str(list(towerdb.columns))
#df = df.rename(columns={'Billing Trigger Date': 'Billing Trigger Date2'})
dic_cols = {'phase msa (phase 1)/pma (phase 2)':'phase _1/_2',\
        'categorization by transmission sys_1' : 'categorization by transmission sys.1',\
        'macro site - transmission hub site with/transmission hub site without shelters': 'macro site - transmission hub site with/without shelters',\
        'transmission sites – with/without shelters.' :'transmission sites - with/without shelters.',\
        'transmission hub sites’': 'transmission hub sites',\
        'easement (servitù di passaggio)': 'easement (servitu di passaggio)',\
        'unnamed: 35':'no_1',\
        'unnamed: 47':'no_2',\
        'rfai ( ready for active installation )' : 'rfai ( ready for active installation ) ',\
        #'diameter\nvodafone antenna': 'diameter vodafone antenna',\
        ' current annual lease fees  ': 'current annual lease fees  ',\
        'energy cost fytd_mar_21': 'energy cost fytd_dec_20',\
        'energy consumption fytd_mar_20': 'energy consumption fytd_dec_20',\
        #'diameter\n(orange rural & unilateranl ' : 'diameter (orange rural & unilateranl',\
        'tlk_1' : 'tlk.1',\
        'rcs&rds_1': 'rcs&rds.1',\
        'bts_site': 'bts sites',\
        'unnamed: 132':'no_3',\
        'unnamed: 134':'no_4'}
towerdb = towerdb.rename(columns=dic_cols)
#towerdb.columns = list(map(lambda x: re.sub(r'^\s+|\s+$', '', str(x.lower())), towerdb.columns.to_list()))

dates_tw =['infrastructure ready (existing)/ to be ready (new)','billing trigger date','first_active_sharing_start_date',
           'first_active_sharing_end_date','radio equipments to be deactivated by', 'rfai ( ready for active installation ) ', 'date_of_equipment_removal']
        
for i in dates_tw:
    towerdb[i] = pd.to_datetime(towerdb[i])
    towerdb[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
                    and not isinstance(date_obj, str) else '' for date_obj in towerdb[i]]
    towerdb[i] = towerdb[i].astype(str)

ints = ['orange passive shared sites ', 'tlk.1', 'orange', 'rcs&rds.1', 'otmos']
for i in ints:
    towerdb[i] = list(map(int, towerdb[i]))

towerdb['no_1'] = ['' for i in range(towerdb.shape[0])]
towerdb['no_2'] = ['' for i in range(towerdb.shape[0])]
towerdb[tw_index] = list(map(str,towerdb[tw_index]))

change_incorret(towerdb, tw_index,'air conditioning','YES','Yes')
change_incorret(towerdb, tw_index,'air conditioning','no','No')
change_incorret(towerdb, tw_index,'air conditioning',0,'No')
change_incorret(towerdb, tw_index,'categorization by site type','DAS Passive','DAS passive')
change_incorret(towerdb, tw_index,'strategic_site_bucket','No','Non Strategic')
change_incorret(towerdb, tw_index,'decommissioned sites','NO','No')

towerdb = towerdb.fillna('')

#'Orange Passive Shared Sites',\
replace_cols = lower_str(['Code',\
             'Phase _1/_2',\
             'Categorization by Transmission Sys',\
			 'Unused sites (TowerCo holds the property rights but does not host any Operator Equipment or Other Customer equipment )',\
			 'Non-Vodafone equipment Sites (TowerCo holds the property rights and does not host any Operator Equipment but hosts Other Customer equipment  )',\
			 'PowerOff Sites',\
			 'Decommissioned sites',\
			 'Categorization by Transmission Sys (sub-cluster)',\
			 'Core Type',\
             'macro site - transmission hub site with/without shelters',\
			 'Transmission sites - with/without shelters.',\
			 'Room Configuration',\
             'Power Supply',\
			 'Air Conditioning',\
			 'Active Sharing Arrangements involving the Operator',\
             'Categorization by Site Type',\
			 'Orange Active Shared Sites ',\
			 '10% Critical',\
			 'Bundled sites',\
			 'Strategic Sites',\
			 'Unilateral Orange Transmission Site',\
			 'First 10 Unilateral Orange Transmission Site',\
			 'Wip_Site',\
			 'Bts Sites',\
			 'Sites_As_Metered_Estimated',\
			 'Strategic_Site_Bucket',\
			 'Subsequent_Sharing_Arrangement',\
			 'Sites_Within_500_Macro_Sites',\
			 'X',\
			 'Site Status',\
             'Critical site'])  
          
for i in replace_cols:
    towerdb[i] = towerdb[i].replace([0, 'N/A', 'n/a', '0', ' '], '')

path_msa = '/content/TowerDB_Romania_20210731.csv'
msa = pd.read_csv(path_msa, encoding='latin').fillna('')
msa.columns = lower_str(list(msa.columns))
msa = msa.rename(columns={
    'phase msa (phase 1)/pma (phase 2)': 'phase _1/_2',
    'transmission sites \x96 with/without shelters.': 'transmission sites - with/without shelters.',
    'transmission hub sites\x92': 'transmission hub sites',
    #'diameter\nvodafone antenna': 'diameter vodafone antenna',
    #'diameter\n(orange rural & unilateranl ' : 'diameter (orange rural & unilateranl',
    'easement (servitù di passaggio)': 'easement (servitu di passaggio)',
    'unnamed: 35':'no_1',
    'unnamed: 47':'no_2',
    'unnamed: 132':'no_3',
    'unnamed: 134':'no_4'})
#msa.columns = [re.sub(r'^\s+|\s+$', '', str(x.lower())) for x in msa.columns.to_list()]
msa.head(3)

towerdb = towerdb[list(msa.columns)]
towerdb = towerdb.rename(columns={
    'no_1':'',
    'no_2':'',
    'no_3':'',
    'no_4':''})

towerdb.to_csv('/content/TowerDB_Romania_20210831.csv', index=False)

In [28]:
from collections import defaultdict
bill_cols = lower_str(['Code',\
             'Phase _1/_2',\
             'Categorization by Transmission Sys',\
			 'Unused sites (TowerCo holds the property rights but does not host any Operator Equipment or Other Customer equipment )',\
			 'Non-Vodafone equipment Sites (TowerCo holds the property rights and does not host any Operator Equipment but hosts Other Customer equipment  )',\
			 'PowerOff Sites',\
			 'Decommissioned sites',\
			 'Categorization by Transmission Sys (sub-cluster)',\
			 'Core Type',\
             'macro site - transmission hub site with/without shelters',\
			 'Transmission sites - with/without shelters.',\
			 'Room Configuration',\
             'Power Supply',\
			 'Air Conditioning',\
			 'Active Sharing Arrangements involving the Operator',\
             'Categorization by Site Type',\
			 'Billing Trigger Date',\
			 'current annual lease fees  ',\
             'Orange Passive Shared Sites ',\
			 'Orange Active Shared Sites ',\
			 '10% Critical',\
			 'Bundled sites',\
			 'Strategic Sites',\
			 'Unilateral Orange Transmission Site',\
			 'First 10 Unilateral Orange Transmission Site',\
			 'Transfer_Date_Of_Registration_Required_Sites',\
			 'Wip_Site',\
			 'Bts Sites',\
			 'Sites_As_Metered_Estimated',\
			 'Strategic_Site_Bucket',\
			 'Subsequent_Sharing_Arrangement',\
             'First_Active_Sharing_Start_Date',\
             'First_Active_Sharing_End_Date',\
			 'Sites_Within_500_Macro_Sites',\
			 'Date_Of_Equipment_Removal',\
			 'RFAI ( Ready For Active Installation ) ',\
			 'X',\
			 'Site Status',\
             'Critical site'])

errors = ['0', '_']

err_dic = defaultdict(list)
for i in bill_cols:
    for value in towerdb[i]:
        if value in errors:
            err_dic[i].append(value)
err_dic

defaultdict(list, {})

In month Checks

In [24]:
"""Check Columns Received""" 
def check_columns_received(df, msa_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in msa_cols if i not in twdb_col]

    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

df_cols = check_columns_received(towerdb, lower_str(list(msa.columns)))
#No columns missing
df_cols

,Column(s) Missing
0,no_1
1,no_2
2,no_3
3,no_4


First Check - Dates Formats (dd/mm/YYYY)

Columns: Date of equipment removal (from MAR´21)

In [ ]:
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            #df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

# Columns to functions
dates_doer = [tw_index, tw_doer]
dates_bill = [tw_index, tw_bill]

#date_parser(towerdb, bill, "%d/%m/%Y", 'no')
#date_parser(towerdb, doer, "%d/%m/%Y", 'mixed')

actives = towerdb[towerdb[tw_status]=='In Service']

no_actives = towerdb[towerdb[tw_status]=='Dismantled']

"""Checking columns for errors"""
actives_dates_errors = check_date_columns(actives, tw_index, tw_status, dates_bill, 2) 
# Actives sites with blank billing trigger date

no_actives_dates_errors = check_date_columns(no_actives, tw_index,tw_status, dates_doer, 2) 
#No errors
print(actives_dates_errors)
print(no_actives_dates_errors)



Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [4]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = df_picklist[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

picklist_tw_general = {
    'categorization by transmission sys': ['0','Long-term Mobile', 'Macro', 'Outdoor Small Cells', 'Public DAS',\
                                           'Repeater', 'Transmission', 'w/o equipment']
}
pick_col_general = ['code', 'categorization by transmission sys']

df_general_pick = check_picklist_v1(towerdb, tw_index, tw_status, pick_col_general, picklist_tw_general)
df_general_pick
#no errors

,site status,categorization by transmission sys
code,,
A5321,WIP,Blank Value
N6548,WIP,Blank Value
N4622,WIP,Blank Value


Fifth Check MoM Sites (BTS, decomissoned...)

In [ ]:
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]

    return filtered[[tw_index,status_col, tw_col]]    

df_mom_bts = check_mom_bts(actives, tw_index,tw_status, tw_bts, msa, msa_index, msa_bts)
df_mom_bts
# No one error df_mom_bts


No Errors Founded!



In [ ]:
def check_mom_bts(df_tw, tw_index,status_col, tw_col, df_msa, msa_index, msa_col):

    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]

    return filtered[[tw_index,status_col, tw_col]]    

decomiss = towerdb[towerdb[tw_decom]=='Yes']
df_mom_decom = check_mom_bts(decomiss, tw_index, tw_status, tw_decom, msa, msa_index, msa_decom)
#No errors


No Errors Founded!



Check Picklist and dates formats for In service sites

In [5]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = df_picklist[df_index]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

picklis_dict = {
    'categorization by transmission sys': ['Long-term Mobile', 'Macro', 'Outdoor Small Cells', 'Public DAS',\
                                           'Repeater', 'Transmission', 'w/o equipment'],
    'categorization by site type': ['DAS passive','GBT','RTT', 'Outdoor Small Cells'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model'],
    'air conditioning': ['No','Yes'],
    'bts sites': ['Yes', 'No'],
    'strategic sites': ['Yes', 'No'],
    'strategic_site_bucket': ['Non Strategic'],
    'critical site': ['Yes', 'No'],
    '10% critical': ['Beyond 10%', 'Within 10%','Non Critical'],
    'wip_site': ['Yes', 'No'],
    'bundled sites': ['Yes', 'No'],
    'decommissioned sites': ['Yes', 'No']
}

picklist_cols = ['Code','Categorization by Transmission Sys','Categorization by Site Type','Sites_As_Metered_Estimated',\
                'Air Conditioning','bts sites','Strategic Sites',\
                'Strategic_Site_Bucket','Critical site','10% Critical','Wip_Site','Bundled sites','Decommissioned sites']
actives = towerdb[towerdb[tw_status]=='In Service']
df_in_service_picklist = check_picklist_v1(actives, tw_index,tw_status, lower_str(picklist_cols), picklis_dict)
df_in_service_picklist
#Tem error em 4 colunas 

,site status
code,


In [ ]:
def check_date_columns(df, df_index,status_col,columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)

    filtered = df[[df_index, status_col]]

    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                value = df_dates.loc[de_site,de_column]
                if date_format.match(value) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                    else:
                        if pd.isnull(value) or pd.isna(value) or value == 'nan' or value=='':
                            df_de.loc[de_site,de_column] = 'Blank Value'
                        else:
                            df_de.loc[de_site,de_column] = f'Incorret picklist value: {value}'
                            
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            df_de.reset_index()
            df_de = pd.merge(df_de, filtered, how='left', on=df_index)
            df_de = df_de[[df_index, status_col, *columns[1:]]]
            return df_de
        else: 
            print('\nNo one columns with incorrect date format!\n')

#check dates in columns
start_dates = ['first_active_sharing_start_date', 'first_active_sharing_end_date']

#date_parser(actives, start_dates, "%d/%m/%Y", 'no')
#actives['First_Active_Sharing_End_Date'] = actives['First_Active_Sharing_End_Date'].fillna('')
in_service_cols = [tw_index, 'first_active_sharing_start_date', 'first_active_sharing_end_date']
df_in_service_dates = check_date_columns(actives, tw_index,tw_status, in_service_cols, 2)
df_in_service_dates
# Varias linhas em branco


Fifth Check BTS Flagged(Billing Trigger and Commercial)

In [9]:
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:
        #df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 
 
actives_1 = towerdb[towerdb[tw_status]=='In Service']
status = 'Yes'
df_bts_empty = check_tw_bill_doer(actives_1, tw_index, tw_bill, tw_bts, status, 'bill')
df_bts_empty
#no errors

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,code,bts sites,rfai ( ready for active installation )


In [13]:
""" BTS sites"""
path_uip = '/content/UserInput_Romania_20210830.xlsx'
uip_names = ['Site_ID','Site Categorization', 'BTS site applicable charge (Annual)',\
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2,3],skiprows=2).fillna('')
uip.columns = uip_names
uip = uip.iloc[1:, :]

msa_sites = [i for i in msa[msa_index]]
tw_sites = [i for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]

Usar as datas no formato Datetime para rodar esse check

In [14]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = pd.to_datetime(df[bill_col], format='%d/%m/%Y', errors='coerce')
    df_site_bts = df[(df[bts_col]=='Yes')&(df[bill_col] > current_date)].fillna('')
    df_site_bts[bill_col] = list(map(lambda x: f'{x:%d/%m/%Y}', df_site_bts[bill_col]))
    df_site_bts = df_site_bts[[tw_index,status_col, bts_col, bill_col]]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

new_sites, df_bts_errors = check_new_sites(towerdb, tw_index, tw_bts, tw_bill,tw_status, msa_sites, tw_sites, uip_sites)

print(new_sites)
print(df_bts_errors)
#No errors

Empty DataFrame
Columns: [New_Sites, site status]
Index: []
Empty DataFrame
Columns: [code, site status, bts sites, rfai ( ready for active installation )]
Index: []


Check WIP Flagged sites


In [15]:
""" Wip Sites Check"""
def check_wip(df_tw,tw_index, wip_tw, tw_bts, tw_status, df_msa, msa_index, wip_msa_col):

    wip_msa = [i for i in df_msa[df_msa[msa_wip]=='Yes'][msa_index]]
    
    tw_wip_sites = [str(i) for i in df_tw[df_tw[wip_tw]=='Yes'][tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]
    
    #wip_out_tw_list = [i for i in tw_wip_sites if i not in wip_msa]
    return tw_wip_site_bts_flagged

df_wip_and_bts_flagged = check_wip(towerdb,tw_index, tw_wip, tw_bts,tw_status, msa, msa_index, msa_wip)

print(df_wip_and_bts_flagged)
# No errors

Empty DataFrame
Columns: [code, wip_site, bts sites]
Index: []


Check Decomissioned sites

In [ ]:
def check_decommissioned(df,df_index,status_col, decom_col, doer_col):
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
  
df_decom_sites = check_decommissioned(towerdb, tw_index,tw_status, tw_decom, tw_doer)
df_decom_sites



Check Doer columns for in service sites

Should not to be in past or different of blank

In [ ]:
#Coluna Doer tem valores fora do formato
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
        
    if t == 'doer':
        df_tw[date_col]= pd.to_datetime(df_tw[date_col], format='%d/%m/%Y', errors='coerce')
        filtered = df_tw[(df_tw[status_col]==status)&((df_tw[date_col] < current_date)|(df_tw[date_col]==''))]
        filtered[date_col] = list(map(lambda x: f'{x:%d/%m/%Y}',filtered[date_col]))
        return filtered[[tw_index, status_col, date_col]] 
    else:

        filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col]=='')]
        return filtered[[tw_index, status_col, date_col]] 

actives = towerdb[towerdb[tw_status]=='In Service']
df_doer = check_tw_bill_doer(actives, tw_index, tw_doer, tw_status, 'In Service', 'doer')
df_doer

,code,site status,date_of_equipment_removal


BTS sites are in subsequent Month

In [ ]:
#Retorna os sites novos
def check_bts(df_tw, bts_tw_columns, tw_index, status_col, df_msa, bts_msa_column, msa_index):

    bts_msa = [i for i in msa[msa[bts_msa_column]=='Yes'][msa_index]]
    tw_bts_sites = [i for i in df_tw[df_tw[bts_tw_columns]=='Yes'][tw_index]]
    #return of datas
    bts_out_tw = [i for i in bts_msa if i not in tw_bts_sites]
    filtered = df_tw.loc[df_tw[tw_index].isin(bts_out_tw), [tw_index, status_col, bts_tw_columns]]
    return filtered

df_bts_out = check_bts(towerdb, tw_bts, tw_index, tw_status, msa, msa_bts, msa_index)
df_bts_out
#No errors



No errors founded!



*Tenth* - Check UIP Towerdb matches

In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col, decom_col, tw_bts_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [str(i) for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]

    uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
    uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                    right_on=tw_index)
    uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]

    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files


    tw_decomiss = [str(i) for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col, decom_col]]

    #Check BTS sites
        #Check BTS sites
    bts_sites = [str(i) for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col, tw_bts_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]

    critical = [i for i in uip_critical if i not in bts_tw_critical]
    critical = pd.DataFrame(critical, columns=['Critical site(beyond 10%) out of UIS'])
    critical = pd.merge(critical, filtered, how='left', left_on='Critical site(beyond 10%) out of UIS',right_on=tw_index)
    critical = critical[['Critical site(beyond 10%) out of UIS', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    
uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                            tw_decom, tw_bts,tw_critical, \
                                                            uip, uip_sites)
print(uis_sites_not_in_towerdb)
print('\n')
print(in_service_not_in_uis)
print('\n')
print(decomiss_sites_in_uip)
print('\n')
print(critical)
 

Commercial

In [ ]:
def check_diffs_v2(path_current, path_last, cols_order, type_file='Excel', sheet='Commercial'):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)

    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')

    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)

path_uis = '/content/UserInput_Romania_20210830.xlsx'
path_before = '/content/UserInput_Romania_20210731.xlsx'
commercial_diffs = check_diffs_v2(path_uis, path_before, '')
commercial_diffs
#Erro na Coluna Param2 Vazio no anterior preenchido no atual

Verificar errors nas colunas 

Creating Excel Log

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

dfs = [['In Service Date Errors', actives_dates_errors],
    ['Dismantled Dates Errors', no_actives_dates_errors]
    ['General Picklist Errors', df_general_pick],
    ['BTS In Month Errors', df_mom_bts],
    ['Decom Dates Errors' df_mom_decom],
    ['In Service Picklist Errors', df_in_service_picklist],
    ['Actives Blank Dates', df_in_service_dates], 
    ['Decom dates Errors', df_decom_sites],
    ['New Sites', new_sites],
    ['New Sites Bill date errors', df_bts_errors],
    ['WIP & BTS Fllaged', df_wip_and_bts_flagged],
    ['DOER Dates Errors', df_doer],
    ['BTS MSA not in TwDB', df_bts_out],
    ['UIS In Month not active', uis_sites_not_in_towerdb],
    ['TowerDB Sites out of UIS', in_service_not_in_uis],
    ['Decom Sites in UIS', decomiss_sites_in_uip],
    ['Critical Sites out UIS', critical]
    ['Commercial Differences' , commercial_diffs]] 

path = '/content/TWDB_RO_errors.xlsx'
general_log_erros(dfs, path)

TA Input

In [53]:
ta_ord = ['Code',
 'Main contract ID',
 'Tenant Agreement (sub contract ID)',
 'Tenant ID (customer ID)',
 'Tenant Name',
 'Classification',
 'Current Annual Fee per Tenant (Annual Hosting (with Discount))',
 'Current Annual Fee per Tenant (Annual Hosting (with Position Discount)',
 'Current Annual Fee per Tenant (Annual Hosting (with Discount)).1',
 'Current Annual Energy Fee',
 'Current Annual Energy Fee.1',
 'Current Annual Maintenance Fee',
 'Current Annual Maintenance Fee.1',
 'Current Other Services Fee',
 'Current Other Services Fee.1',
 'Starting date',
 'Expiring date',
 'Renewal Option',
 'Expiring date after renewal',
 'Terms of payments',
 'Payment type',
 'Index',
 'Percentage',
 'Indexation driver',
 'VAT Subject',
 'Percentage (VAT)',
 'Termination Date']

pathtw = '/content/TowerDB_Romania_20210831 (1).xlsx'
sheet= 'Tenant Template'
ta_dates = ['Starting date', 'Expiring date', 'Expiring date after renewal']
ta_amount = ['Current Annual Fee per Tenant (Annual Hosting (with Discount))']
ta_bill = ['Tenant Name','Classification','Starting date', 'Expiring date']

#ta.columns= [re.sub(r'^\s+|\s+$', '', str(x.lower())) for x in ta.columns.to_list()]
ta = pd.read_excel(pathtw, sheet_name = sheet, skiprows=7)
ta = ta.iloc[:,2:]
#df.columns= [re.sub(r'^\s+|\s+$', '', str(x.lower())) for x in df.columns.to_list()]
ta = ta.dropna(subset=['Tenant Name'], axis=0)
ta['Current Other Services Fee.1'] = ['' for i in ta['Code']]
ta['Current Annual Fee per Tenant (Annual Hosting (with Discount)).1'] = ['' for i in ta['Code']]
ta['Current Annual Maintenance Fee.1'] = ['' for i in ta['Code']]
ta['Current Annual Energy Fee.1'] = ['' for i in ta['Code']]
ta = ta[ta_ord]

for i in ta_dates:
    ta[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) \
             and not isinstance(date_obj, str) else '' for date_obj in ta[i]]

"""df = replace_values(df, cols_amount, 0)
for col in cols_amount:
    df[col] = df[col].fillna(0)
    df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')"""
"""df = replace_values(df, cols_int, 0)
for col in cols_int:
    df[col] = df[col].fillna(0)
    df[col] = df[col].astype(int)"""

for i in ta_bill:
    ta[i] = ta[i].replace(['N/A', 'n/a',"0", np.nan,'nan'], '')
#date_parser(df, cols_date, format, type_date)

ta = ta.fillna('')

ta = ta.rename(columns={'Current Annual Fee per Tenant (Annual Hosting (with Discount)).1': 'Current Annual Fee per Tenant (Annual Hosting (with Discount)).',\
                        'Current Annual Energy Fee.1': 'Current Annual Energy Fee.',\
                        'Current Annual Maintenance Fee.1': 'Current Annual Maintenance Fee.',\
                        'Current Other Services Fee.1': 'Current Other Services Fee.'})

ta.to_csv('/content/TA_Input_Romenia_20210831.csv', index=False)


In [ ]:
ta[ta_dates]

# Check for empty fields or negative differences

In [ ]:
import datetime as dt

#Create dates in datetime format
ta['end'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in ta['Expiring date']]
ta['start'] = [pd.to_datetime(i, format='%d/%m/%Y') if i!= '' else datetime(1999, 1, 1, 0, 0, 0) for i in ta['Starting date']]
#Create diff columns to get negative values
ta['dt_dff'] = (ta['end'] - ta['start']).dt.days

start_or_end_blank = (ta['Starting date']=='')|(ta['Expiring date']=='')|(ta['Expiring date after renewal']=='')
negative_dates = (ta['dt_dff'] <= 0)
cols = ['Code','Tenant Name', 'Starting date', 'Expiring date', 'Expiring date after renewal']
ta_dates_errors = ta[start_or_end_blank|negative_dates][cols]

ta_dates_errors

In [ ]:
ta =ta.drop(columns='end')

# Check for empty Classification or Incorrect Values

In [ ]:
df_class_error = ta[(ta['Classification']=='')|~(ta['Classification'].isin(['MNO', 'OTMO']))][['Code', 'Tenant Name', 'Classification']]
df_class_error

# Log creation

In [ ]:
def general_log_erros(df_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')  
    for lista in df_list:
        try:
            if not lista[1].empty:
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
        except AttributeError:
            pass
        except:
            if lista[1] is not None and lista[1].is_valid():
                lista[1].to_excel(writer, sheet_name=lista[0], index=False)
    try: 
        writer.save() 
    except IndexError:
        print('No one log to write!')

tas = [['Picklist Errors', df_class_error],
       ['Dates Errors',df_class_error]]

path = '/content/TA_RO_errors.xlsx'
general_log_erros(tas, path)